In [ ]:
%cd ..

In [ ]:
import os
import numpy as np
from molecule.utils.utils_notebook import get_MI_df, get_ranked_df, LATEX_FIG_PATH, get_DTI_rank_df, process_dataset_name, prerpocess_emb_name
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import clear_output

METRIC = r"$\overline{\mathcal{I}_S}$"
DATASET = "ZINC"
results_dir_list = ["run_4"]
COLUMS_SPLIT = "cond_modes"
df = get_MI_df(DATASET, results_dir_list)
df_downs = get_ranked_df(df, path="results/TDC_ADMET_SCAFF.csv")
df_downs_pre_task_type = get_ranked_df(df, split_on="task_type", path="results/TDC_ADMET_SCAFF.csv")
clear_output()

# Heatmap of performances on downstream tasks in communities

In [ ]:
def plot_cmap_comm_tasks(
        df,
        df_downs,
        res,
        cols = "dataset",
        row="metric normalized",
        method = "ward",
        communities = [{'ChemBertMLM-77M', 'ChemBertMLM-5M', 'ChemGPT-1.2B', 'MolBert', 'ChemBertMLM-10M', 'ChemGPT-19M', 'ChemGPT-4.7M'}, {'MolR_gat', 'ChemBertMTR-5M', 'ChemBertMTR-10M', 'FRAD_QM9', 'MolR_gcn', 'ChemBertMTR-77M', 'DenoisingPretrainingPQCMv4', 'MolR_tag'}, {'GraphCL', 'GraphLog', 'GROVER', 'ThreeDInfomax', 'Not-trained', 'InfoGraph', 'AttributeMask', 'GraphMVP', 'ContextPred', 'GPT-GNN'}, {'MoleOOD_OGB_GCN', 'MoleOOD_OGB_GIN', 'MoleOOD_OGB_SAGE'}],
        save=True
    ):
    df_downs["dataset_id"] = df_downs["dataset"] + df_downs["run_id"].apply(str)
    df_piv_min = df_downs.groupby([cols, "embedder"]).mean().reset_index().groupby(cols).metric.min()
    df_piv_max = df_downs.groupby([cols, "embedder"]).mean().reset_index().groupby(cols).metric.max()
    df_piv_mean = df_downs.groupby([cols, "embedder"]).mean().reset_index().groupby(cols).metric.mean()

    df_piv =  df_downs.copy().groupby([cols, "embedder"]).metric.mean().to_frame()

    df_piv["metric normalized"] = (df_piv["metric"] - df_piv_min) / (df_piv_max - df_piv_min)
    df_piv["metric centered"] = df_piv["metric normalized"] - df_piv_mean
    df_piv = df_piv.reset_index()


    df_piv["community"] = df_piv["embedder"].apply(lambda x: [i for i, community in enumerate(communities) if x in community][0])

    g = sns.clustermap(
        df_piv.pivot_table(
            index="embedder",
            columns=cols,
            values=row,
            aggfunc="mean"
        ),
        cmap="coolwarm",
        figsize=(10,10),
        center = 0.5 if row == "metric normalized" else 0 if row == "metric centered" else None,
        method=method,
    )
    # name of embedders following the community
    cmap_com = sns.color_palette("dark", len(communities))
    for i, label in enumerate(g.ax_heatmap.get_yticklabels()):
        text= label.get_text()
        com = df_piv[df_piv.embedder == text].community.values[0]
        color = cmap_com[com]
        label.set_color(color)
        label.set_weight("bold")


    #remove row dendogran
    g.ax_col_dendrogram.set_visible(False)
    #remove colorbar
    g.cax.set_visible(False)
    #rotate x labels
    plt.setp(g.ax_heatmap.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor")
    #remove x and y labels
    g.ax_heatmap.set_xlabel("")
    g.ax_heatmap.set_ylabel("")
    plt.xlabel("")
    plt.ylabel("")
    if save:
        plt.savefig(
            f"{LATEX_FIG_PATH}/molecule/communities_task_{method}.pdf",
            format="pdf",
            bbox_inches="tight",
        )

In [ ]:
%matplotlib inline

plot_cmap_comm_tasks(df, df_downs, 1, method="ward")


In [ ]:
%matplotlib inline

plot_cmap_comm_tasks(df, df_downs, 1, method="ward", cols="dataset_id", save=False)
